# Flexible Universal Character Level Encoding method for Text in NLP

    Leonardo M. Rocha
    leo <dot> m <dot> rocha <at> gmail <dot> com
    

## Abstract

Currently NLP deals with *Out of Vocabulary* (OOV) in different ways, this leads to several non-necessarilly efficient ways of pre-processing NLP datasets to be able to deal with 
In this work we present **Segment-Multihot-Encoding** (SME) a technique that deals with OOV and allows to encode all possible symbols in a computationally efficient manner to encode all (or part) of the UTF-8 domain in a fixed multi-hot encoding that can be further compressed by overfitting. This technique eliminates the need of complex and compute consuming pre-processing replacing it for a much more simple one that works for *any* dataset. This work focuses on being able to encode a symbol, as once it is encoded the network can be fine-tunned later to handle previously unseen ones.

This work presents the SME technique for UTF8 and we call it **SME-UTF8**, the source code and encoding vectors are also given and examples are shown in other notebooks.

## Introduction and Related Work

Currently for NLP tasks there is the need to first analyze the input domain and encode it to deal with *Out of Vocabulary* (OOV) words or symbols and Polysemy. 

It is important to separate (and we do in this work) the encoding part (to be able to represent the symbol) from the learning to use those symbols (the network to be able to do something useful with it) as this work focuses solely on being able to encode all feasible symbols in a defined text encoding domain. In this case the work is done for UTF-8 which is the most used text encoding in the web [94.6% according to w3techs](https://w3techs.com/technologies/details/en-utf8).



Diverse techniques deal differently with OOV, ranging from techniques that can not deal with them, like [GloVe - Pennington et al. 2014](https://nlp.stanford.edu/pubs/glove.pdf) or [Word2Vec - Milikov et al. 2013](https://arxiv.org/abs/1301.3781) to others such as [Universal Sentence Encoder -Cer et al. 2018](https://arxiv.org/abs/1803.11175) or the one for FastText can encode OOV with subword embeddings
Several papers use word level encoding plus character level ngrams

[ELMo - Peters et al. 2018](https://arxiv.org/abs/1802.05365)

[Transformer - ]()
[ULM-FiT - ]()
[BERT - ]()
[AlBERT - ]()
[CamemBERT]()

... TODO more papers and references here

All 

All current methods deal with subdomains of the possible inputs available, which for most tasks is enough, nevertheless the weakness is that they can not deal with **all** posisble input symbols, which for the current study is UTF-8.

In the case of continual learning the need to add new symbols is a given, be it due to adding new domain in the same language, or new languages

The goal of this work is to try to set a more standard way to deal with all possible symbols in a defined encoding standard.

This work analyzes UTF-8 encoding and presents a technique to be able to encode part or all of the UTF-8 domain in an computationally efficient way. The same technique can be used for other text encodings without any modification, and as utf-8 is a superset of other encodings (ASCII, Unicode, ...) the same matrix encoding can be applied without any modification in those datasets.

There is also another point to say about this computational complexity, all the current SoTA methods are trained in clusters (and wiht prices) that are unavailable to most users. The current work is part or a larger work on trying to get enough permormance in commercially available (and relatively accessible) single GPUs for end users (being at the current time the NVidia RTX2080ti one of the more computationally strong cards in the market).


## UTF-8 Analysis

### One-Hot encoding

[One-hot encoding](https://en.wikipedia.org/wiki/One-hot) is one of the most used to encode cathegorical variables, in the case of State of the Art NLP tasks is used to encode the input symbols, this is computationally expensive and the goal here is to reduce this complexity leaving memory and computational space for other more complex tasks in the network.

### Number of code-points

As this file tries to encode all the characters possible by utf-8 we have to check the feasible number so:

From [Wikipedia utf-8](https://en.wikipedia.org/wiki/UTF-8)

UTF-8 is a variable width character encoding capable of encoding all **1,112,064**.

$$17×2^{16} = 1114112 $$ code points minus 2,048 technically-invalid surrogate code points

This is, if encoding with one-hot we would need 1.1M parameters per neuron in the input layer, which is expensive. The goal is to reduce this complexity (which we argue is unnecessary) by orders of magnitude.

## UTF-8 structure and 

Since the entire utf-8 univers is NOT the entire $2^{32}$ domain, but there are limitations explained in [the utf-8 description](https://en.wikipedia.org/wiki/UTF-8)

| Number of bytes | Bits for code point | First code point | Last code point | Byte 1   | Byte 2   | Byte 3   | Byte 4   |
|----------------|--------------------|-----------------|----------------|----------|----------|----------|----------|
| 1              | 7                  | U+0000          | U+007F         | 0xxxxxxx |          |          |          |
| 2              | 11                 | U+0080          | U+07FF         | 110xxxxx | 10xxxxxx |          |          |
| 3              | 16                 | U+0800          | U+FFFF         | 1110xxxx | 10xxxxxx | 10xxxxxx |          |
| 4              | 21                 | U+10000         | U+10FFFF       | 11110xxx | 10xxxxxx | 10xxxxxx | 10xxxxxx |

The UTF-8 code is formed by 4 segments, we will refer to this often during the current work.

The thing is that the number of elements in the table should be at most $2^{21}$, There is only the need to create a index that can handle the 4 cases which can be done with 4 different conversion tables.


In fact it is possible to just cut the utf-8 value in chunks and do one-hot per different parts:
- there are only 4 segment ranges, that can be coded to add redundancy in one-hot also add there either hamming or other ECC
- the largest value is for 7 bits -> 128 values
- the others contain 6 bits -> 64 values
The prefix of each can be taken away and replaced by the initial one-hot

So a complete code would be:  $ 4 + 128 + 64 + 64 + 64 = 324 $

Instead of having a vector of dimension 1,112,064 to encode any utf-8 value, one with dimension 324 (even 320) would be able to encode everything.

This embedding can stil be reduced but should be sparse enough already to make a good input, the goal here is to have sparse vector that makes each vector far enough of the others, at least by one dimension. Adding the redundancy code (the first 4 dimensions) allows to make distance even bigger for vectors that should be appart. 

It is worth noting here that the first author has also experience in communications which allowed during the curse of this research the analysis of multiple Error Correcting Codes (ECCs) and different kinds of encoding (for example encoding as a Fourier Series), the conclusion is that even if the one-hot is the best in distance, other codes can be used and a multi-hot sparse is the simplest to implement (and fastest to encode). As a note, one pending task is to analyze ECCs in an end to end manner for a neural network.